# First submission! Just submit this preliminary work to see if the scoreboard works ;)

More to come...

## Shopee Project - Determine if two products are the same by their images

The general workflow is:

### Image
* Stacking two backbone models for transfer learning: ResNet50 pretrained on MS-COCO (downloaded from the Internet) + ResNet50 pretrained on ImageNet (from Keras) => 
* Supervised feature selection by ANOVA => 
* Agglomerative clustering by cosine similarity

### Title
* Tokenize => Indonesian Stemmer + WordNet Lemmatizer => 
* Calculate TF-IDF =>
* Clustering

Merge the results from the image cluster and title cluster

In [ ]:
from shutil import copytree
copytree('../input/zzxjoanw/hdbscan-0.8.27/hdbscan-0.8.27','./hdbscan')

In [ ]:
%cd hdbscan

In [ ]:
!python setup.py bdist_wheel

In [ ]:
%cd dist

In [ ]:
!pip install "hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl" --no-cache-dir --no-binary :all: --no-build-isolation
import hdbscan

In [ ]:
%cd /kaggle/working

In [ ]:
# from nltk.cluster.kmeans import KMeansClusterer
# from nltk.cluster.util import cosine_distance

In [ ]:
# !pip install "../input/zzxjoanw/nltk-3.6.2-py3-none-any.whl"

In [ ]:
import numpy as np
import pandas as pd

import os

In [ ]:
import struct

# Image tools
#import imgaug

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation, Average, Concatenate, Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

import scipy
from sklearn import *

from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

# NLP tools
from html import unescape
from nltk.util import everygrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input as ppi_vgg16, decode_predictions as dp_vgg16
from keras.applications.resnet50 import ResNet50, preprocess_input as ppi_resnet50, decode_predictions as dp_resnet50
from keras.utils import plot_model

from joblib import load

# from glob import glob
# from shutil import copy2
from gc import collect

In [ ]:
# for glob('/kaggle')

In [ ]:
import cv2
import numpy as np

def preprocess(path):
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 205, 255, cv2.THRESH_BINARY_INV)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

    small1 = []
    small2 = []
    large1 = []
    large2 = []

    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
#         if w*h < 1000:
#             continue
        small1.append(y)
        small2.append(x)
        large1.append(y+h)
        large2.append(x+w)

    dst = img[np.min(small1):np.max(large1), np.min(small2):np.max(large2)]

    s = max(dst.shape[0:2])

    f = np.full((s,s,3),255,np.uint8)

    ax,ay = (s - dst.shape[1])//2,(s - dst.shape[0])//2

    f[ay:dst.shape[0]+ay,ax:ax+dst.shape[1]] = dst

    f = cv2.resize(f, (300, 300))
    
    return f

In [ ]:
# Indonesian stemmer

!pip install '../input/zzxjoanw/PySastrawi-1.2.0-py2.py3-none-any.whl'
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
df0 = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
# df0

In [ ]:
testpaths = df0['image'].tolist()
titles = df0['title'].tolist()

In [ ]:
from gc import collect
collect()
mapping = {
"acne": ["jerawat"],
"acrylic": ["akrilik"],
"acoustic": ["akustik"],
"adult": ["dewasa"],
"aluminum": ["aluminium"],
"arm": ["lengan"],
"artificial": ["artifisial"],
"baby": ["bayi"],
"backpack": ["ransel"],
"rucksack": ["ransel"],
"balloon": ["balon"],
"bath": ["mandi"],
"shower": ["mandi"],
"battery": ["baterai"],
"bh": ["beha"],
"belt": ["sabuk"],
"big": ["besar"],
"bike": ["sepeda"],
"biscuit": ["biskuit"],
# "black": ["hitam"],
"book": ["buku"],
"bottle": ["botol"],
"bra": ["beha"],
"bracelet": ["gelang"],
"broom": ["sapu"],
"brush": ["sikat"],
"bulb": ["bohlam"],
"bulumata": ["bulu", "mata"],
"butter": ["mentega"],
"cable": ["kabel"],
"caffeine": ["kafein"],
"calculator": ["kalkulator"],
"camera": ["kamera"],
"candle": ["lilin"],
"car": ["mobil"],
"automobile": ["otomobil"],
"capsule": ["kapsul"],
"cardigan": ["kardigan"],
"carpet": ["karpet"],
"case": ["kasus"],
"cheap": ["murah"],
"child": ["anak"],
"chocolate": ["cokelat"],
"coffee": ["kopi"],
"coklat": ["cokelat"],
"collagen": ["kolagen"],
"color": ["warna"],
"comb": ["sisir"],
"computer": ["komputer"],
"concentrate": ["konsentrat"],
"conditioner": ["kondisioner"],
"contents": ["isi"],
"cream": ["krim"],
"detergent": ["deterjen"],
"disinfectant": ["desinfektan"],
"dinosaur": ["dinosaurus"],
"discount": ["diskon"],
"drug": ["obat"],
"dryer": ["pengering"],
"dumbel": ["barbel"],
"dumbell": ["barbel"],
"dumble": ["barbel"],
"electric": ["elektrik"],
"eraser": ["penghapus"],
"extract": ["ekstrak"],
"eyelash": ["bulu","mata"],
"fabric": ["kain"],
"facemask": ["masker"],
"fan": ["kipas"],
"faucet": ["keran"],
"feet": ["kaki"],
"flower": ["bunga"],
"folding": ["lipat"],
"fridge": ["kulkas"],
"funny": ["lucu"],
"glass": ["kaca"],
"glue": ["lem"],
"guitar": ["gitar"],
"hair": ["rambut"],
"hairdryer": ["pengering","rambut"],
"pengeringrambut": ["pengering","rambut"],
"hand": ["tangan"],
"handbag": ["tas", "tangan"],
"hanger": ["gantungan"],
"hat": ["topi"],
"helmet": ["helm"],
"honey": ["madu"],
"hose": ["selang"],
"inch": ["inci"],
"jacket": ["jaket"],
"jamtangan": ["jam", "tangan"],
"japan": ["jepang"],
"juice": ["jus"],
"keyboard": ["papan","ketik"],
"papanketik": ["papan", "ketik"],
"kitchen": ["dapur"],
"knife": ["pisau"],
"lamp": ["lampu"],
"lash": ["bulu mata"],
"leather": ["kulit"],
"lipstick": ["lipstik"],
"long": ["panjang"],
"lychee": ["leci"],
"margarine": ["margarin"],
"man": ["pria"],
"men": ["pria"],
"mens": ["pria"],
"mascara": ["maskara"],
"mask": ["masker"],
"material": ["bahan"],
"metal": ["logam"],
"metre": ["meter"],
"micro": ["mikro"],
"microphone": ["mikropon"],
"microwave": ["mikrowave"],
"milk": ["susu"],
"month": ["bulan"],
"mosquito": ["nyamuk"],
"motive": ["motif"],
"necklace": ["kalung"],
"nipple": ["puting"],
"nylon": ["nilon"],
"oil": ["minyak"],
"package": ["paket"],
"pant": ["celana"],
"paper": ["kertas"],
"pearl": ["mutiara"],
"cooker": ["penanak","nasi"],
"penanaknasi": ["penanak", "nasi"],
"pencil": ["pensil"],
"pillow": ["bantal"],
"plastic": ["plastik"],
"polyester": ["poliester"],
"porcelain": ["porselen"],
"printer": ["pencetak"],
"quartz": ["kuarsa"],
"rack": ["rak"],
"raincoat": ["jas", "hujan"],
"jashujan": ["jas", "hujan"],
"ring": ["cincin"],
"robe": ["gamis"],
"rope": ["tali"],
"sash": ["selempang"],
"sauce": ["saus"],
"scissor": ["gunting"],
"screwdriver": ["obeng"],
"semprotan": ["semprot"],
"shampoo": ["shampo"],
"shoe": ["sepatu"],
"short": ["pendek"],
"silicon": ["silikon"],
"silicone": ["silikon"],
"soap": ["sabun"],
"sock": ["kaus", "kaki"],
"soy": ["kedelai"],
"soya": ["kedelai"],
"kauskaki": ["kaus", "kaki"],
"spray": ["semprot"],
"sprayer": ["semprot"],
"sticker": ["stiker"],
"t-shirt": ["kaos"],
"tea": ["teh"],
"telephone": ["telepon"],
"tennis": ["tenis"],
"tent": ["tenda"],
"thermometer": ["termometer"],
"tomato": ["tomat"],
"tool": ["alat"],
"toothbrush": ["sikat","gigi"],
"towel": ["handuk"],
"transparent": ["transparan"],
"sikatgigi": ["sikat", "gigi"],
"toothpaste": ["pasta","gigi"],
"pastagigi": ["pasta", "gigi"],
"tissue": ["tisu"],
"towel": ["handuk"],
"toy": ["mainan"],
"tshirt": ["kaos"],
"umbrella": ["payung"],
"vacuum": ["penyedot","debu"],
"penyedotdebu": ["penyedot", "debu"],
"vanilla": ["vanila"],
"wall": ["dinding"],
"wallet": ["dompet"],
"warranty": ["garansi"],
# "white": ["putih"],
"woman": ["wanita"],
"women": ["wanita"],
"womens": ["wanita"],
"wool": ["wol"],
"year": ["tahun"]
}

def flatten(list_of_lists):
    if len(list_of_lists) == 0:
        return list_of_lists
    if isinstance(list_of_lists[0], list):
        return flatten(list_of_lists[0]) + flatten(list_of_lists[1:])
    return list_of_lists[:1] + flatten(list_of_lists[1:])

# TF-IDF as clustering dimension

tt = TweetTokenizer()
wnl = WordNetLemmatizer()
ignored = stopwords.words('english')
ignored += stopwords.words('indonesian')

def tokenize(desc):
    desc = unescape(desc).replace('/',' ').replace('\\',' ').replace('T-','t').replace('t-','t').replace('-',' ').replace('_',' ').replace("'",'').replace('h&m','hnm').replace('H&M','hnm')
#     ret = [x for x in [stemmer.stem(wnl.lemmatize(wnl.lemmatize(tok,'v'),'n')) for tok in tt.tokenize(desc) if tok not in ignored and tok.isascii() and tok.isalpha()] if len(x) > 2]
    ret = [x for x in [stemmer.stem(wnl.lemmatize(wnl.lemmatize(tok,'v'),'n')) for tok in tt.tokenize(desc) if tok.isascii() and tok.isalpha()] if len(x) > 2 or x.lower() == 'bh' or x.lower() == 'hp']
    
    ret = [mapping[x] if x in mapping else x for x in ret if ret not in ignored]
    ret = flatten(ret)
#     ret = list(everygrams(flatten(ret),max_len=2))
#     ret = [x for x in ret if ret not in ignored]
    
#     print(ret)
    return ret

tfidf = feature_extraction.text.TfidfVectorizer(tokenizer=tokenize, use_idf=True, max_features=40000)
X_trainx = tfidf.fit_transform(titles).toarray()

# len(tfidf.vocabulary_)

# pengukur -> ukur; pemutih -> utih; penggorengan -> goreng; etc.

In [ ]:
lsa = decomposition.TruncatedSVD(n_components=750, random_state=0)
lsa.fit(X_trainx[0:50000])
X_trainx = lsa.transform(X_trainx)

In [ ]:
from gc import collect
collect()
# mapping = {
# "acne": ["jerawat"],
# "acrylic": ["akrilik"],
# "acoustic": ["akustik"],
# "adult": ["dewasa"],
# "aluminum": ["aluminium"],
# "arm": ["lengan"],
# "artificial": ["artifisial"],
# "baby": ["bayi"],
# "backpack": ["ransel"],
# "rucksack": ["ransel"],
# "balloon": ["balon"],
# "bath": ["mandi"],
# "shower": ["mandi"],
# "battery": ["baterai"],
# "bh": ["beha"],
# "belt": ["sabuk"],
# "big": ["besar"],
# "bike": ["sepeda"],
# "biscuit": ["biskuit"],
# "black": ["hitam"],
# "book": ["buku"],
# "bottle": ["botol"],
# "bra": ["beha"],
# "bracelet": ["gelang"],
# "broom": ["sapu"],
# "brush": ["sikat"],
# "bulb": ["bohlam"],
# "bulumata": ["bulu", "mata"],
# "butter": ["mentega"],
# "cable": ["kabel"],
# "calculator": ["kalkulator"],
# "candle": ["lilin"],
# "car": ["mobil"],
# "automobile": ["otomobil"],
# "cardigan": ["kardigan"],
# "carpet": ["karpet"],
# "case": ["kasus"],
# "cheap": ["murah"],
# "child": ["anak"],
# "chocolate": ["cokelat"],
# "coffee": ["kopi"],
# "coklat": ["cokelat"],
# "color": ["warna"],
# "comb": ["sisir"],
# "computer": ["komputer"],
# "contents": ["isi"],
# "cream": ["krim"],
# "detergent": ["deterjen"],
# "disinfectant": ["desinfektan"],
# "dinosaur": ["dinosaurus"],
# "discount": ["diskon"],
# "drug": ["obat"],
# "dryer": ["pengering"],
# "dumbel": ["barbel"],
# "dumbell": ["barbel"],
# "dumble": ["barbel"],
# "electric": ["elektrik"],
# "extract": ["ekstrak"],
# "eyelash": ["bulu","mata"],
# "fabric": ["kain"],
# "facemask": ["masker"],
# "fan": ["kipas"],
# "faucet": ["keran"],
# "feet": ["kaki"],
# "flower": ["bunga"],
# "folding": ["lipat"],
# "fridge": ["kulkas"],
# "funny": ["lucu"],
# "glass": ["kaca"],
# "glue": ["lem"],
# "guitar": ["gitar"],
# "hair": ["rambut"],
# "hairdryer": ["pengering","rambut"],
# "pengeringrambut": ["pengering","rambut"],
# "hand": ["tangan"],
# "handbag": ["tas", "tangan"],
# "hanger": ["gantungan"],
# "hat": ["topi"],
# "helmet": ["helm"],
# "honey": ["madu"],
# "hose": ["selang"],
# "inch": ["inci"],
# "jacket": ["jaket"],
# "japan": ["jepang"],
# "juice": ["jus"],
# "keyboard": ["papan","ketik"],
# "papanketik": ["papan", "ketik"],
# "kitchen": ["dapur"],
# "knife": ["pisau"],
# "lamp": ["lampu"],
# "lash": ["bulu mata"],
# "leather": ["kulit"],
# "lipstick": ["lipstik"],
# "long": ["panjang"],
# "lychee": ["leci"],
# "margarine": ["margarin"],
# "man": ["pria"],
# "men": ["pria"],
# "mens": ["pria"],
# "mascara": ["maskara"],
# "mask": ["masker"],
# "material": ["bahan"],
# "metal": ["logam"],
# "metre": ["meter"],
# "micro": ["mikro"],
# "microphone": ["mikropon"],
# "microwave": ["mikrowave"],
# "milk": ["susu"],
# "month": ["bulan"],
# "mosquito": ["nyamuk"],
# "motive": ["motif"],
# "necklace": ["kalung"],
# "nipple": ["puting"],
# "nylon": ["nilon"],
# "oil": ["minyak"],
# "package": ["paket"],
# "pant": ["celana"],
# "paper": ["kertas"],
# "pearl": ["mutiara"],
# "cooker": ["penanak","nasi"],
# "penanaknasi": ["penanak", "nasi"],
# "pencil": ["pensil"],
# "pillow": ["bantal"],
# "plastic": ["plastik"],
# "polyester": ["poliester"],
# "porcelain": ["porselen"],
# "printer": ["pencetak"],
# "rack": ["rak"],
# "raincoat": ["jas", "hujan"],
# "jashujan": ["jas", "hujan"],
# "ring": ["cincin"],
# "robe": ["gamis"],
# "rope": ["tali"],
# "sash": ["selempang"],
# "sauce": ["saus"],
# "screwdriver": ["obeng"],
# "semprotan": ["semprot"],
# "shampoo": ["shampo"],
# "shoe": ["sepatu"],
# "short": ["pendek"],
# "silicon": ["silikon"],
# "silicone": ["silikon"],
# "soap": ["sabun"],
# "sock": ["kaus", "kaki"],
# "kauskaki": ["kaus", "kaki"],
# "spray": ["semprot"],
# "sprayer": ["semprot"],
# "sticker": ["stiker"],
# "t-shirt": ["kaos"],
# "tea": ["teh"],
# "telephone": ["telepon"],
# "tennis": ["tenis"],
# "tent": ["tenda"],
# "tomato": ["tomat"],
# "tool": ["alat"],
# "toothbrush": ["sikat","gigi"],
# "towel": ["handuk"],
# "transparent": ["transparan"],
# "sikatgigi": ["sikat", "gigi"],
# "toothpaste": ["pasta","gigi"],
# "pastagigi": ["pasta", "gigi"],
# "tissue": ["tisu"],
# "towel": ["handuk"],
# "toy": ["mainan"],
# "tshirt": ["kaos"],
# "umbrella": ["payung"],
# "vacuum": ["penyedot","debu"],
# "penyedotdebu": ["penyedot", "debu"],
# "vanilla": ["vanila"],
# "wall": ["dinding"],
# "wallet": ["dompet"],
# "warranty": ["garansi"],
# "white": ["putih"],
# "woman": ["wanita"],
# "women": ["wanita"],
# "womens": ["wanita"],
# "wool": ["wol"],
# "year": ["tahun"]
# }

# def flatten(list_of_lists):
#     if len(list_of_lists) == 0:
#         return list_of_lists
#     if isinstance(list_of_lists[0], list):
#         return flatten(list_of_lists[0]) + flatten(list_of_lists[1:])
#     return list_of_lists[:1] + flatten(list_of_lists[1:])

# # TF-IDF as clustering dimension

# tt = TweetTokenizer()
# wnl = WordNetLemmatizer()
# ignored = stopwords.words('english')
# ignored += stopwords.words('indonesian')

# def tokenize(desc):
#     desc = unescape(desc).replace('/',' ').replace('\\',' ').replace('-',' ').replace('_',' ')
# #     ret = [x for x in [stemmer.stem(wnl.lemmatize(wnl.lemmatize(tok,'v'),'n')) for tok in tt.tokenize(desc) if tok not in ignored and tok.isascii() and tok.isalpha()] if len(x) > 2]
#     ret = [x for x in [stemmer.stem(wnl.lemmatize(wnl.lemmatize(tok,'v'),'n')) for tok in tt.tokenize(desc) if tok.isascii() and tok.isalpha()] if len(x) > 2 or x.lower() == 'bh']
    
#     ret = [mapping[x] if x in mapping else x for x in ret if ret not in ignored]
#     ret = flatten(ret)
# #     ret = list(everygrams(flatten(ret),max_len=2))
# #     ret = [x for x in ret if ret not in ignored]
    
# #     print(ret)
#     return ret

# tfidf = feature_extraction.text.TfidfVectorizer(tokenizer=tokenize, use_idf=True, max_features=15000)
# X_trainx = tfidf.fit_transform(titles).toarray()

# # len(tfidf.vocabulary_)

# # pengukur -> ukur; pemutih -> utih; penggorengan -> goreng; etc.

In [ ]:
# tfidf.vocabulary_

# 'victoria secret'
# 'marks spencer'
# 'speedo'
# 'quicksilver'

In [ ]:
# lsa = decomposition.TruncatedSVD(n_components=200, random_state=0)
# lsa.fit(X_trainx[0:50000])
# X_trainx = lsa.transform(X_trainx)

In [ ]:
# # TEST BLOCK
# df1 = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
# # qqq = df1[df1['title'].str.contains("sexy")]
# # plt.imshow(Image.open(f'/kaggle/input/shopee-product-matching/train_images/{qqq["image"].tolist()[0]}'))
# # trainpaths = qqq['image'].tolist()
# df1

In [ ]:
# %%time
# preprocess(f'/kaggle/input/shopee-product-matching/test_images/{df0.iloc[0]["image"]}')

In [ ]:
# !ls

In [ ]:
from tensorflow.python.keras.applications.efficientnet import EfficientNetB3

In [ ]:
model_b4_imagenet = EfficientNetB3(weights='/kaggle/input/zzxjoanw/b3_goodman_.h5', include_top=False, input_shape=(300,300,3))
tt = model_b4_imagenet.get_layer('top_activation').output
avg_pool = GlobalAveragePooling2D(name='avg_pool')(tt)

model = Model(inputs=model_b4_imagenet.input,
          outputs=avg_pool)
# print(model_b4_imagenet.summary())
ins = np.zeros((0,1536))
batch = []

images = np.zeros(shape=(1, 300, 300, 3))

img = preprocess(f'/kaggle/input/shopee-product-matching/test_images/{testpaths[0]}') 

x_expand = np.expand_dims(img, axis=0)
images[0, :, :, :] = x_expand

batch.append(images)
for path in range(1,len(testpaths)):
#     start = time.time()
    images = np.zeros(shape=(1, 300, 300, 3))

    img = preprocess(f'/kaggle/input/shopee-product-matching/test_images/{testpaths[path]}') 
    
    x_expand = np.expand_dims(img, axis=0)
    images[0, :, :, :] = x_expand
    
    batch.append(images)
    if path % 64 == 0:
        batch = np.vstack(batch)
        images_features = model.predict(batch, batch_size=64)
        batch = []
        ins = np.vstack([ins,images_features])
if path % 64 != 0:
    batch = np.vstack(batch)
    images_features = model.predict(batch, batch_size=path % 64)
    batch = []
    ins = np.vstack([ins,images_features])

In [ ]:
# model_resnet50_conv = ResNet50(weights='/kaggle/input/shopee-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224,224,3))

# tt = model_resnet50_conv.get_layer('conv5_block3_out').output
# avg_pool = GlobalAveragePooling2D(name='avg_pool')(tt)

# model_resnet50_conv = Model(inputs=model_resnet50_conv.input,
#           outputs=avg_pool)

# # print(model_resnet50_conv.summary())

# weights = np.load('/kaggle/input/shopee-weights/weights.npy', allow_pickle=True)
# weights = weights.tolist()

# for layer in model_resnet50_conv.layers:
#     if layer.name in weights:
# #         print('Set!')
# #         try:
#         layer.set_weights(weights[layer.name])
# #         except:
# #             pass
# #             print(layer.name)#, layer.get_weights())
#     layer._name += '_'
# del weights
# # collect()

In [ ]:
# #TEST BLOCK

# '''
# model_resnet50_imagenet = ResNet50(weights='/kaggle/input/shopee-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)#, input_shape=(336,336,3))
# tt = model_resnet50_imagenet.get_layer('conv5_block3_out').output
# avg_pool = GlobalAveragePooling2D(name='avg_pool')(tt)

# model_resnet50_imagenet = Model(inputs=model_resnet50_imagenet.input,
#           outputs=avg_pool)


# for layer in model_resnet50_imagenet.layers:
#     layer._name += '_'
# #     layer.trainable = False

# # for layer in model_resnet50_conv.layers: 
# #     layer.trainable = False


# last_layer_resnet50 = model_resnet50_conv.get_layer('avg_pool').output
# last_layer_resnet50_imagenet = model_resnet50_imagenet.get_layer('avg_pool_').output

# concatenate = Concatenate()([last_layer_resnet50_imagenet, last_layer_resnet50])

# model = Model(inputs=[model_resnet50_imagenet.input, model_resnet50_conv.input],
#           outputs=concatenate)

# print(model.summary())
# model.compile()
# model.fit()
# '''

# ###################################################

# df1 = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
# qqq = df1[df1['label_group'] == 2544002498]#.str.contains("01ef80c9ba5f247d2fcbb0edffa624e1.jpg")]
# plt.imshow(Image.open(f'/kaggle/input/shopee-product-matching/train_images/{qqq["image"].tolist()[0]}'))
# trainpaths = qqq['image'].tolist()

# #VGG16

# # model_vgg16_conv = VGG16(weights='imagenet', include_top=True)
# model_resnet50_conv = ResNet50(weights='imagenet', include_top=True)
# # print(model_vgg16_conv.summary())

# # print(type(model_vgg16_conv.get_layer('fc2').output))
# # model = Model(inputs=model_vgg16_conv.input,
# #           outputs=model_vgg16_conv.get_layer('fc2').output)

# # for layer in model_vgg16_conv.layers: 
# #     layer.trainable = False
# for layer in model_resnet50_conv.layers: 
#     layer.trainable = False
    
    
# # print(model_vgg16_conv.summary())


# # last_layer = model_vgg16_conv.get_layer('fc2').output
# # # out = Flatten()(last_layer)
# # out = Dense(2048, activation='relu', name='fc3')(last_layer)


# last_layer_resnet50 = model_resnet50_conv.get_layer('avg_pool').output


# # out = Flatten()(last_layer)
# # out_resnet50 = Dense(1000, activation='relu', name='fc4')(last_layer_resnet50)

# # averaging = Average()([out, last_layer_resnet50])

# model = Model(inputs=model_resnet50_conv.input,
#           outputs=model_resnet50_conv.output)



# # print(model.summary())

# ins = []

# for path in trainpaths[0:19]:
#     images = np.zeros(shape=(1, 224, 224, 3))

#     img = Image.open(f'/kaggle/input/shopee-product-matching/train_images/{path}')
#     if img.mode != 'RGB':
#         img = img.convert('RGB')

#     img = img.resize((224, 224), Image.NEAREST)

#     x_raw = image.img_to_array(img)

#     x_expand = np.expand_dims(x_raw, axis=0)
#     images[0, :, :, :] = x_expand

# #     inputs = ppi_vgg16(images)
#     inputs_2 = ppi_resnet50(images)
    
#     images_features = model.predict(inputs_2)
    
#     ins.append(images_features[0])

#     print(len(ins))
# #     print(images_features)

# # vector = images_features[0]
# # print(vector.shape)
# # # (4096,)
# # print(vector)
# # # [0.        0.        0.        ... 0.        1.9249601 0.8309637]

In [ ]:
# #TEST BLOCK 2
# from keras.applications.resnet50 import decode_predictions
# index = 1

# print('Predicted:', decode_predictions(np.array([ins[index]]), top=10))

# plt.imshow(Image.open(f'/kaggle/input/shopee-product-matching/train_images/{trainpaths[index]}'))
# print(qqq.iloc[index]['image'])

In [ ]:
# mycatalog = {
# 679: ['7d9eb586807596ec2ca874cf71958543.jpg','094f820ccae44c168b369a55ef7426bb.jpg','a53e3c9d4988ab112a0916fd1bff1edd.jpg','1f498ccff290f4b0faf1670a5df152b7.jpg','0c7e3b17589af7dd503bd425a205ccc1.jpg', '3a491861eb274af64012cf74b60fc936.jpg', '4ac591da7985d8eccbcbd8f03975d9ad.jpg', '2245192a363df2ef3b9b4063b2da30e7.jpg'], #'necklace'
# 629: ['179a696af375f4e854e05df8255ee892.jpg','645bd436e1ec36a215343931c0a8a1c5.jpg','d14cc3e0e5f6d7abcc25aaecee1c679c.jpg','5f014c1c69d457012d4f8d718b54a58d.jpg','de485fa2218ec76ab8d0491de1fee825.jpg','e0697e13d4c1c4a0f11df1c415a842a6.jpg','f5a47aded059d05f8b274fc3cbc44937.jpg','f8e982d4381c2afbbbf075971639cb20.jpg','bfbfdc2e279d75cc1361b3f27cb9c63d.jpg','ab6955a7deaa5bcaeda638c78c4242c8.jpg','9372003783ed1b413ec05505f0e49d84.jpg', 'a0b1e800a73cc89fab7f043ca91cfe54.jpg', 'e6919fab97ccb2fa59f6661937a81f0d.jpg', '900955305c12861180dc5943b2750be8.jpg','935032991e638de1b5ca2da4b0ddde8f.jpg','1319df13cb1021f8bdfb0c24efb5ab0b.jpg','167820ecd2d088dc9d5022a9e9be5d89.jpg','55fd9bc5488fe4884fdbbbad23d45a22.jpg', '20da4838a048f2d23b34a6894e9ebecc.jpg', '9501dbb2cadeb3a6bead86b86cedd8ca.jpg'], #'lipstick'
# 721: ['1e233aeefbab5dc40cb82bced4a6e10b.jpg','2956402da81f90afcd4e3d278fbc1c62.jpg'], #'pillow'
# 529: ['2e0b9dda50452825c62510476aa3629d.jpg'], #'diaper'
# 838: ['056abe07b8611a723043fa76e3cd29dc.jpg'], #'sunscreen'
# 531: ['4e23883a50066354aa2e2c583a991c0c.jpg','4b1f31be2f71ef65b034c4946b7b57cb.jpg','ea0415b296607a1964383b6b20e31ab8.jpg','f2e38d4cb42125ab00c55b9400c76959.jpg','ef9c0d37414bb4c5c209243198b34cdd.jpg','ef0360cba9e9aaa25889607ec6ecb4b1.jpg','b7933f3365646388b5bb57ded08ceff5.jpg','31d798cf9cb3afd50b15953af092ee8b.jpg', '2ef4e0c080f6247e9189911011b696ee.jpg', '29afb0c56c7c78fbf745185ce782dbd8.jpg', '2265562d3faa59a14820848c47035210.jpg', 'c08db46ff2b4e018c63810f5e26a1996.jpg', 'c018c7ccf97eebfeee7a769e9e5698e8.jpg', 'd03ecc97e497fd6a2495fc1e82b5d9e4.jpg', 'ef9c0d37414bb4c5c209243198b34cdd.jpg', 'ef0ecfff4b090769efff1279e4a130d7.jpg'], #'digital_watch'
# 872: ['f0b1a186ff676deadc5184459eb7c175.jpg', 'eacdb554f6093a0a37e7926d72e45431.jpg','965d6610bc5b87ea019f863b2a1a5f43.jpg','7eed9547efd1d563f1e90b1e21273e06.jpg','7ceaeafbfce3e49351db2aea747df401.jpg','92775acfd30fed5dd626e86d256112e9.jpg','c3a70c8fc6e6800c6edae4bc5987ebe1.jpg','b1dd38fc24572ae4b3a53302b4333e41.jpg','f05e95952bf8040c4d9c6531f9870765.jpg','b5002f4f9f459fe9bfd272d929d41a53.jpg','0773fb57a009303d68b3a7f1a5ab8cd1.jpg', '05319116819e35785440f4946559af7e.jpg', '0449f7437f5534f0643db1446809948d.jpg', '0804ea0a607f64a45e3f47ed07ec4b24.jpg', '0ab4b6d52b4cc8ef858858878c4ef47a.jpg', '13787d4d6078d050e72ced991291da15.jpg', '1dab201966404bc62a46aba04f8efdb4.jpg', '36e04e7c8361ea3304ecb897d4aca075.jpg', 'a2aae0dbfc21e293805f3bf98dd9c85b.jpg', 'c9c5cd498fef2cf5333b58a609b638e8.jpg', '666b751f6d415d92e44fe7427aeb54ad.jpg','0773fb57a009303d68b3a7f1a5ab8cd1.jpg'], #'tripod'
# 608: ['11b7a4efe6c116acbf3815ff8f11909e.jpg','54a3de9e28884edd577c4e52e9e80221.jpg','51296d17142b41e4149c5d4ca75b1112.jpg','ccd142b65682b7994fa72027020822e5.jpg','6c3e5ca7be73fe21c8c990d386b62baf.jpg','4b87f9a74b15eb6b73ee3d163a1698af.jpg','e21d87a301a473a3fb1d1f93de6fb274.jpg','e15e27a971ea7e6b22e8fc53d9b2a185.jpg','e78bbccfefbbe11cdc3cec0bb78644d7.jpg','c2ca61cff7e79bff3aa73b588beefe8e.jpg','bf96ae7a3d6d2f38b10cfe03abddf4b0.jpg','658626690eb8105b8917071f35772025.jpg', '22adf09260786ed124339d46f7f74c16.jpg', '2717080e6a297cab37944a1abb87dc48.jpg', '210251403be1834cc81fb5be25af7d42.jpg','f6450e1d4481236b86e1e748c1954cad.jpg','f333d3c0f6afce624c6d62f9e0949a47.jpg', 'c831046f841fa7510587d90b92779977.jpg', '79ca03d0d7de4da582cb70e09282dec3.jpg'], #'jean'
# 774: ['302503554d6b589d3c3e0970dfb60d8a.jpg','55f064604e4faa8751bf20baad9af8ff.jpg','571cc66830ad61454f796c4565481cd9.jpg','cd4bb44e0dfc394c8cef8dfa2fe1de72.jpg','c3580167fa178e29688bc0c1268a5831.jpg','b0fc18946dd11e6e2d84d60300dd6178.jpg','bf69ead3dadbf3ebe850b1d34314eafe.jpg','4f1809715669554376a275d0d1452879.jpg','4e22370f385f048c6f582c6a979f24d4.jpg','3b44667811b051457307c21e598884d7.jpg','dd19b7373adce902481aa6196e9cf85f.jpg','e4f34986d40d58d266040d4a5ae6b74c.jpg','e5c60113d9c45a0656bec397384ce925.jpg','e318ae0c3867c38bcb6e3bac970b5241.jpg','eaa49d2fc9875a2f924d28f2c5369a50.jpg','f0333494b559e600acdf1583256e337b.jpg','fa05675d4e40749f4f7a49769ad03e88.jpg','0a17608e8933b475089f8cc4cd1f05b3.jpg','bfa36ab9b30b62f808c5f33dda030105.jpg','ba4cfa4daf5cbfbdc0396f4a159309aa.jpg','b07dce524e238769d98363d04a3b9b5d.jpg','af160323509e5fdbca9088c0b102e6d6.jpg', 'ac9efe99a864e7b5a2705c43a5c883a2.jpg','ab3451b1f2b1776eaed2b3746e2a240e.jpg','ac2bbb871156d7ed0b388e801c46a21b.jpg','aa6e56e2588037eecbd767563d05c9d0.jpg','a142472d7c2ae55f766a4ec9f372a190.jpg', 'a24803e3e454188337e04f94ef338f13.jpg','839012532bdb784e94dc7e6140364a44.jpg','86613916c4c4593fc27dc0d15e44b290.jpg', '2777638c8ced5a3abcee5f2ddbca1e9e.jpg', '28347927145e3db04609685a5b208523.jpg', '55f064604e4faa8751bf20baad9af8ff.jpg', '94c9efff22e041a6f4e4a66f235438cf.jpg', '04399b5e0278bc4696ff0ff600abb1be.jpg', 'd41879577b388348b655c498782b8888.jpg', '6264747cf982b325fc8eda606fa9a917.jpg','42926335eb60e2d40e0ea9833c6ee1c4.jpg','127180009071e792606d746704713e55.jpg', 'c916dc5f48bf34f6379f9b3a19c93b27.jpg', 'cafe5887ee8265c752253d86661bb51b.jpg', 'eeaa777e2f5b735305b5365dd30a11fa.jpg','739f630255e57a8e54e65163adeda8f5.jpg'], #'sandal'
# 841: ['82d77bb94decaa458aa41c838a1dfb01.jpg','5fa8baef22e0d4ffd14e990245c3afd8.jpg','5be34c4cb7e29a362cd91c00916e79ac.jpg','468f84f0f89a7c6ac9a877e3153798a2.jpg', '0e0d6836b2e06ee164038cd25211797d.jpg', '5823ad9b77c57d1c4775bdc9e0659701.jpg', '78125afc64af1e69bbb0d1f72525911e.jpg', '143632cfd07032b7591d9bfe8c4fb678.jpg','9ead37b4042ab2ce3f653519616a9f8e.jpg'], #'sweatshirt'
# 459: ['1359a8a3dad5b8325ddd00385c5e62ba.jpg', '7f37e09997e0b35b2cb1cfde94d517d4.jpg','8e0a8cf994c172fe4d1ef87a78f403c8.jpg','21f274e54981771174a6fbd636e93093.jpg','4796d3bb74b2735cfd551a1a67f7fdd9.jpg','22095d2abdc4e3f4089e45fcf62c0744.jpg','22458d067913415207c3a7b1bcead1ba.jpg','48061dbbef7896e2d32a78f198c4b6b1.jpg','d9ee64997addc1f4c9eef7b9da86fe06.jpg','b6c31356ac15935fd1012ec71bea06fa.jpg','ad5c3757fb4f14e2fb5437654c3a6754.jpg','a63ca037b2e9d53c3afa3afb9009de40.jpg','6b5ea21577fbe7510a3dcbab66588e58.jpg','5f9db1032b981d0a17bc191d6b4ba194.jpg','5ff2187a74405f65bda9f1ac032dba86.jpg','5dbc1eeffa4464ba2e1bbfad889e24ab.jpg','3bb97a9579b9642cb3c99a4ef7a27b88.jpg','2d931937c77674880148540e9f28f06b.jpg','1e230d03f1255de231e01304914b22bd.jpg','df9ceb3df7cd0c51290b01d5089229c5.jpg','e0c3d16205282d09885e5c670daea8c4.jpg','e29cc53cc2cb08afe1fe21d570653beb.jpg','e80fa25c9feb0d3775ae0166b240514a.jpg','e886278ce3ada280b9ee0a12332dd1bf.jpg','eb78b627ddb4a7978105286cf1db97fc.jpg','eb87465a079ab14db5febee3a9267af9.jpg','ff7542f679ba6b7a7c0841c8d33fc5f8.jpg','0c820c826c416cb1c8b330e4f4c80747.jpg','b6ab08a3bcf128f86f26f32ba83e5aba.jpg','ab5ffedf44263beed14dafc04ec7a3a3.jpg', '9857861c1e456febf189d44eea98e849.jpg','3596818a14bb02905101f0b50bda197c.jpg', '2750507f46680c6f1e76119ca6aaa5c6.jpg', '6460bd87b8ce3de050648553604d676e.jpg', '1a9b5b146caa8e7daa5c1f6a73587ae5.jpg','0d282f9acfda90977384c131c84d6628.jpg', '00c982e628bbc4dfed595fa941a920d5.jpg', '3cb43a3d05b87c7eded53262685cef6c.jpg', '3f4bae5be5b0723b2f40ef95bc1dad90.jpg', '3e4661f2bf9854692b1ed2ba57f42c21.jpg', '3dbafef9f292893b2870af6bf27ff5a1.jpg', '6f163dcdec1ff920ad843fd4c467977a.jpg', '8e7deec5df5a35c5f68805d9d9d41590.jpg', '020e241595d433e247c906ef303e2bc7.jpg', '437a77842d486f81467178ebc85efe2c.jpg', '418c6ed67dae65c6e25b6998f795aa4b.jpg','92f5aa2dbea16c926c5fb1735d62d08f.jpg', '96ec95c63f4c2111e4ed563041091cb0.jpg', '165f4794206411da8dffe1fcbd096787.jpg','db65188fa5cce03daecc4c9fa54d33e4.jpg', '386400bc30e61107a9eb93d44c08b02f.jpg','a1c598045dab2e06b5ea0fffe2db926d.jpg','a63ca037b2e9d53c3afa3afb9009de40.jpg','a651116db76fb88f1384428007b0033c.jpg','e644602a1ba208885b174f0d738cd6a3.jpg', 'edd56b8da05544bcff004cc43d57d4ec.jpg','3596818a14bb02905101f0b50bda197c.jpg'], #'brassiere'
# 879: ['0a772c4c0ee5b7a3352c6225c28a2155.jpg', '7e7600266c846444fcbf7b0ce20d191d.jpg','7c530411b58d6f804fca3b77e519262e.jpg','d58c81788347fd4b20ce595a7cc62632.jpg','d33f0e4eca0cab3f7c5be3ae9a39596b.jpg','76858002195dbf6694fa5dfb05311ff8.jpg','91452812610ec41f87f3c0863ca2b077.jpg','7c530411b58d6f804fca3b77e519262e.jpg', '504f37735c02990623bdea9fa8d900f5.jpg', '603907b049a655f26995ce9d18e8fbeb.jpg','b622781ba40588ae1fc06a185338e00f.jpg', 'bdcc0213c64a33829408fc9464ecf3e0.jpg', 'c7e2270dbe1703536257804c402f1ab9.jpg', 'ee074eb8828b8d2c24482d87e3fe2955.jpg', '746ea2eaeed3e10f619d100a62078622.jpg', '76858002195dbf6694fa5dfb05311ff8.jpg'], #'umbrella'
# 770: ['578d4d9928fb0ae85946987f2003f3b6.jpg','2aced07ecd531b61852987941fbb73b4.jpg','1d7eca78e7f0df9652851e6b4a246e96.jpg', '2d64fd1d319a246189c7994c824a367c.jpg','9bacf85167c2338a4c20b18cccd68236.jpg','9bcbc9219d0f38013d7d2056a8d1e499.jpg','18fd0c0664d801652c8ce893490eb6d1.jpg','194ca4bb6f47674060d1863649033059.jpg','302f8f2cccf13cf7d19dcfed3fd9e214.jpg','0894b3d0e5621943a22b3a11645f7ba4.jpg','ce76603d4d9013a02fbda09ce18e70d4.jpg','cb2260db75abe73c689319673c9200f9.jpg','bc50e18da4507121d8725285de9d2170.jpg','a962bea97971068e3727531ab20f7c32.jpg','a701d722bd3ff02975bc851a21ff959d.jpg','a30b3c610b0026d19a6e8f236d22782e.jpg','a5e9b5081b302878613c3474a460cbd8.jpg','6eec026b7300c3088f7441e3d0c482c9.jpg','4eefef790d1233410fae0f569b1ee06b.jpg','4f604bf1521d4a03ebf709c6ac0da55d.jpg','4f2d012ff2e98e3416eb02f9686fe414.jpg','4bd8f0697b19d9b90e9b19b5a5d42d9d.jpg','3a5a9eb12fc9ddb2de291d183684174b.jpg','2d55107ca3a3dd01a2f5e1dd6199862e.jpg','2d64fd1d319a246189c7994c824a367c.jpg','2b67ffdabde13d65fe05f064c6ff1223.jpg','dcd2090a68eecffb8ee2b210bf404a32.jpg','e1e8013b2c4b88377d6b621a3a3f2f3d.jpg','e67e1ae0fea65dc5181e746aec3273d4.jpg','ea5c00b5b05b8ae8a23bba1a782e2dcd.jpg','ec084c28d824476591dd41ce63decef3.jpg','ed8e9a201903734583e8dc3cbb7516c8.jpg','ef74bde2fd61acc369057395f8592068.jpg','f7eb265244261574e2c2d26e94e84439.jpg','f6d879c6e283c904823983706eadb48a.jpg','f4f67dc88c791d04177cbc952936777f.jpg','f315e1281bd0ec976b0b7e3bbd7128e2.jpg','f6884de51a219cfbd86f9bddea41b20b.jpg','f606188b7bcc6877cdbdd6dc23399d5a.jpg','fc3e15bba5447dbdc1e3caa2ec66f98b.jpg','c05ed86dc0760753595af9b87bef4f45.jpg','bfa43cb953822ac155d2c302c69ab697.jpg','be739c79f957ecc13c16edc68cbc3e00.jpg','b923e0afba666121fa5d171ec8636f72.jpg','b991ccaf20c116e6c7da3fd265439688.jpg','b97e70750a455b57cae51e135c1d9fbd.jpg','b03de193cbbfa207650658f028ee348a.jpg','aff936534db08d2b8bcef53a98cb941f.jpg','afff64e7a28066573e383724425f721b.jpg','ad7bda034ae0d6066dde5b3964fd9308.jpg','ac207c613389d198ac134fcd2767e38a.jpg','2035511750db2d0e38655c87bffb54b3.jpg','774315845acb5c3da5103a1d69ba5880.jpg','12648678f395a5922343ccf8d583321f.jpg','0601660f4866ac2234e69f2228173dc1.jpg', '0be368f2a859d04773e900ddff36de09.jpg', '0de1f2dee4d00c869e756dc33c6a13db.jpg', '022f08b9f59000f83743d955ea75cd6a.jpg', '8e2bc9ba73f6c91ad4f095a66a73371a.jpg', '8e8e3094b3053c85306c7075fe309e3f.jpg', '57e9ca22c3a521665355a58ed55fcf6c.jpg', '75a6fd196541b74aa4bdbcaae36b8543.jpg', '75c3ae81c8548fa0804f3b5dac867197.jpg', '74fa3d472dddc55214dfb2033dc74c74.jpg','93ddd65aaa750a220c15ba04b92f7edf.jpg', '99d0bc9d673ecdf992bae327f12985a0.jpg', '4126f69927142b0770cc194997fdf92f.jpg', '44862e9f04bcaf8d2060c9ce0a608387.jpg', '494754d81d1cb5e5ebc95076fa21cece.jpg', '5353818cab8501b55d5b44a52c928d55.jpg', '9183733dbd5eb6ceceface1c21e78e60.jpg','16393719963d72c57cb166b280e007b1.jpg','a8c9a71c152166d3c1efad1c36f18716.jpg','a213db7809b1e439ef56a5535f8b4569.jpg','a5177d6251872211f0ac1ad201c89e6b.jpg','eee73f2a89bc9629f10eb77c57023720.jpg', 'c178e5f11d509bb5097da60d08795843.jpg', '832ff1fcedd866d62a6baa907f0eafcb.jpg'], #'running_shoe'
# 850: ['3b4bbf52bfd419d12a5ece198c158360.jpg','68ed39246edc71e3e7cdc895260b172d.jpg', '76a1fde7ea0f0e7c9b329eb2aeb8d39b.jpg','34335882f21c7413a32c802731fc6bc1.jpg','a196cf823ee2a7074ed05f1ac1bd11b4.jpg', '194069ad1ae0bb42e94528f27ad20218.jpg'], #'teddy'
# 534: ['9356119b273b5910543bb4f79a2532b8.jpg','b0b62367c397877b93480fe6c295abf1.jpg'], #'dishwasher'
# 817: ['8294763d8ea34f2cdd746d445c7f2f1d.jpg','33116a82ccb68558eb0c8d39ba63ada6.jpg'], #'sports car'
# 543: ['64918528aa011e5e7c0ee1ac4951f961.jpg','aa0434efecaa2f6d279b1cfe730d2265.jpg','6eec026b7300c3088f7441e3d0c482c9.jpg', '3ae23b785676a6678cba41716968d53b.jpg', '66e6b635d96ec181f5a51f33cf08776d.jpg'], #'dumbbell'
# 414: ['1c6058733e0fe1140b631f6f2395dacc.jpg','7c7cdda3d07f899437a527241d7bd280.jpg','50ffe9c1b54c786540d3322bc2cbd791.jpg','96c1753ee2e221f16fb8fe372801d517.jpg','857c5c67e497bd39fec3b4657de13f3d.jpg','913e7eb462b7ef493f871a2556ab5efb.jpg','a6227bb09acf3b22a4d6281dd4cf2b48.jpg','a4911e8297d4df35d204cb0a20f70ecf.jpg','a284c25ef8358f698fe6f8b15351096a.jpg','06f7022abf0285cfe0c61d7ecb21c1e6.jpg','2eeaeb80df88558ef8b7ab30f164933a.jpg','e6a77c715554569fccf3d4cd0a24738f.jpg','e27a1a12e8917998ecbb6051ce828fa5.jpg','efba0d9a6234a78526b0b01acf5a2849.jpg','f528ebd248e8c92356ff40cf6e6811c9.jpg','fa8071dde0ab781c9232968498d13f87.jpg','0baac87a6caefaec5536ec082b847348.jpg','bf4d7852f9064ad41a9b8cd6b7a2e216.jpg','b15597a6c2f9ace93c4af87f799f8b71.jpg','b093ef6895f4118921791a29b8f20ecd.jpg','ac503e7921a39a4c6508f0951b4cd60a.jpg','89152993fadda6b99d4cd477fcce7364.jpg','4034486608b9c61968d5ef8f919aade4.jpg','a6227bb09acf3b22a4d6281dd4cf2b48.jpg','ab08abc48c1c88963b4edd88a4abb08d.jpg', 'e5f1423ed6218320aea91beb5f888d4d.jpg', '544ea45f27ce6722eb5d88dcaf0d5e26.jpg'], #'backpack'
# 545: ['bb3d6e5761e9a7d5492c969babc3de19.jpg','1cd23c1d7246d93d6d1a8aaab00f03df.jpg', '2cd58c3d4cb9747942e1e75f7ae666c9.jpg','7a71e1fa793cc8279de9495b443a4258.jpg','14ae6912e4f77919c45d5bd1ce529cf6.jpg','21fd77a5e19e95031e1aae48006f96c7.jpg','25ecc4e4494495522c9be64444fca87a.jpg','196de96131fe1643a674967a3c4065b0.jpg','685dae43b1033f4762ac433cd798bb2a.jpg','05294ac1c3211c5d3f0c29d938142f41.jpg','73814c276b72cf2bfaa44450b5f9be0b.jpg','5806723d4d60b2da6d8d107cfacc3c98.jpg','c5ba556763012def2dac8661d1b3e62f.jpg','a9aa475c62f2e74e06a74ec733849209.jpg','4c72448801e13a466e6a6f07c0f9e493.jpg','2cd58c3d4cb9747942e1e75f7ae666c9.jpg','df33a8b9ce1c27b69e2bb51f89387286.jpg','bb3d6e5761e9a7d5492c969babc3de19.jpg','457909670eb28fa218a7c5e308521ebc.jpg','697018763618ab36f4661b84b7242171.jpg','b2271996569880c8976a9ab4b958d896.jpg', 'bcdeb28679842279fc8f9d02fa6bd276.jpg','8910bdece5c35792a4171f568a7e44b1.jpg'], #'electric fan'
# 478: ['51572173b2a6f1bbdcedfa175da8bbcf.jpg','0ff783a9cb9bf53dd1fdc06abd0f3dff.jpg', '6b8431174ccfa314320b75f88c9331a2.jpg','05551b3ce4892481b74bad64c8a73b5a.jpg','a645eab9c88429837885d7875923c42c.jpg', 'b13dfbf26ae24b599ecf428388338612.jpg','ede1bde493f13fe17988fdcb4c7b410d.jpg','5fdaab57e0fd4fdb0f8c8c7fb9b90c82.jpg', 'eb5ce2d842534d6b6db84be943ee98bb.jpg'], #'carton'
# 409: ['1490777d0246c968da8c1d3da5b62ea7.jpg', '391acabf1fd9614190870d5a7176b2e5.jpg','ad5e66b3baca2093a9aa4db2f635fec0.jpg','4c159d412d7a60d22d3eb91004ae7353.jpg','3b6b2ec1515f09e0f501d2664903eea5.jpg'], #'analog clock'
# 662: ['b4dff4c9f8f3471784eecf5d9bf16f28.jpg','0702432226b85eda034d54993c356316.jpg','6b4f39de15d5f26b5c552db4e9c34150.jpg','1bf2da06c71aec2ed16fd5daaccbd1c6.jpg','1def9f82392b9278baf87f4310ef72e6.jpg'], #'modem'
# 567: ['116b77f454e271d4ffcf790a187dbcd4.jpg','1d22e9bd8b9e311898834577dc0e3b6b.jpg', '8c2398b68014afa67bdfe54dda35c952.jpg','023bd583a216f4d833b292829c2912e0.jpg','003524b70715bf6bfa00451ca08e66e0.jpg','69912149ccd7bc4ab25ec6fbd7b0afaf.jpg','b9c8b1a9cf989b97b39b499f30efeb2c.jpg','cd882b23e548ba9a5f7cc1338645edd7.jpg', 'ef40415fc55e9816d22995075edfea3e.jpg', '67ad224f692ecfee7ee1197867010804.jpg', '68a9ca597a75fd0c54205ab96faf1edd.jpg'], #'frying pan'
# 473: ['b14664fa5f20164b00d9b91dd5a348ac.jpg','fadc4d24a0527e1c7643cc0be1e38bb3.jpg','ec8d0738c14f6f9ea6ddc4fd1c14ee9e.jpg'], #'can opener'
# 589: ['a8df9e0f08529a0c275bd3d8e5cead53.jpg','beda4792779229e497984652adbb5e5a.jpg','b08ae04de32980a3b53dfb249cd1bd02.jpg','b6086f04f01fd1a9c9e0d7be90903a5f.jpg','ee3c5ef6298d97ce0f7ad24bb8a1004e.jpg','e709e4739fa95dc25cc0124e1e380cd8.jpg', 'ee3c5ef6298d97ce0f7ad24bb8a1004e.jpg'], #'hand blower'
# 680: ['735c22c487503cecfb8029149f615731.jpg','7c6293f9f746fdae3f8554e6ef62695a.jpg','0195cc4b21fdd80c8c7d810bb42b72d6.jpg','580e1bab850844db4326417fb4373dcb.jpg','54947fc4968c8f7bfe2db33d6a87ecf3.jpg','cac57159594e20c27b52fa74a9df9f68.jpg','c48da7ed5f071a643ada1e66ea82f3cd.jpg','2e39a64a52113ee0dbf292a56226b0a1.jpg','0f6c77ffcece7520c6704f41237b02b7.jpg','00d97097eeeb5fb5929562d70b1c9928.jpg','e6b05a20d7f91bf479da544d3d67a95e.jpg','ba690273464d73c5a1f4dc75c9b00f0f.jpg','f96015ecbb7edc5c5a1717978f3d91b9.jpg','e1bfc87a3317a92b2aac929b0ba3ee92.jpg','dfd6a25ec37e9e7f9f4afb981f86ce44.jpg','44bce17a72c602720d18ad72b5803dc8.jpg'], #'nipple'
# 898: ['2ead2b6ac1f7d9109517c9d030bf9fd4.jpg', '03952dc36e0e61eb6f25442414bc806d.jpg','ba124cc38bb1eaafcf77f3c8ae48f998.jpg', 'edf4bde4d013b25d69b6be5c92a0b6b3.jpg', '680c90955d8f7527187d113c2e14f00d.jpg'], #'water bottle'
# 544: ['2a14a2bd4c62dcc23f40aad2201f7dd0.jpg','be1bd213c21ed7b1854732e68b48d564.jpg','a8fc2f4dc1ea69b1385d14c18944c3f9.jpg'], #'dutch oven'
# 606: ['962651d6f91debeeb678c346c7dad036.jpg','a06a6b71d60c2b5f8dcb6ddbad35b25d.jpg','6abd4c6b0d8b58ac21638f6de4ded987.jpg','fa9aa9917783d3265629842ca2fd07bb.jpg','ecb2d307fb274677bd26163791b8670b.jpg','9915270307eebe3477eaeeb10b307830.jpg','119b00f72a1e5ca97d13d6c9366d5bde.jpg', 'd766143e4ec29cf7e986419d1ecc753a.jpg'], #'iron'
# 430: ['f5414db8c330c336be58dc8064417f44.jpg','10c6a05de7947e97874d200aea0fd6dd.jpg','6a69b5c33ab9a9ccf02d51786ac90587.jpg','a9cf7d71b02d106257f40171eccd117d.jpg', 'd5361cb1c29ebfad53808f20026c96d6.jpg'], #'basketball'
# 632: ['e6e60998fb24dbbe52850a0bf737e1d3.jpg','3deba9781d29d22bb0e04f62cf3567da.jpg','3879c8f495be10ccef43a1f1e5004533.jpg'], #'loudspeaker'
# 610: ['0a68f2c8b6eab3352c602dfc7fea9e54.jpg'], #'jersey'
# 518: ['bae7493d5137e70e2c3b2f048016b7e2.jpg','2d09655e6313b6ec07d59f1a5eeb4251.jpg','4d137c73de3f420b9524971a296c0b93.jpg','5e272dc53b945fbfa2804751bda425fc.jpg'], #'crash helmet'
# 673: ['169f53dccfd343038246cdd56eed7135.jpg', '2a3d7bf7819b0a4258dc53adff5b2b1a.jpg','4eea48583ec4dd6c5ff964b6b287883d.jpg','4f2d012ff2e98e3416eb02f9686fe414.jpg','44c8def4d34ea14d2d6591655a35d279.jpg', '334e33bb8a9f07b3e546cb22c2eae529.jpg'], #'computer mouse'
# 620: ['bf241eff75d74b79bbcebfc2cdb2c3d4.jpg','31dda21fa1e06fd8d40cb4d04c3f21bd.jpg'], #'laptop computer'
# 697: ['7c9f759b8a17bb819961b161c7342eb7.jpg','43a17fdbce001a79c9d98adc4197a0fc.jpg','d3212cb492aea55b6dba4b8edc872b00.jpg','ce7c00fd0976639cb783bca370403760.jpg','bb21dd250d0dbbdac19a57c5986c2114.jpg','a5f6bc46e19135941e0106642bc1a89d.jpg','ab5f56be0243533695706bf62780bd77.jpg','01f40752b0bfd610a5f2df27c31f4e10.jpg','313efddc507378040c853dbcc740ebca.jpg','313efddc507378040c853dbcc740ebca.jpg','4ef5026a4d50b6c737c51d594b46146d.jpg','a4839ec6c036aa92f6bec901acb2db27.jpg','3c1abf50df502b6aadcec47fc23e2ab3.jpg','3bbdf30bf83e1e9e25b4f42f012c6863.jpg', 'ecfac1eb4bdf24bb9f86bff75af02fcd.jpg','d3175c417cb07997a768ef9fec04b5a9.jpg'], #'pajama'
# 672: ['9d8ae2a1b2c5a0c1ddc7ad225ecf79fe.jpg','b9b1eb8bd1ee544269f69cd1b61edb49.jpg','6827f27272c0862a78bec9340117e6b1.jpg','a48ab19a9b70f8e56175331f40721909.jpg','3801248af1687cf5dbfe04828a2b763d.jpg'], #'mountain tent'
# 479: ['b26e210dfdb34d6980df0d6acb61b7e4.jpg','6c95d557f905d1a5addcf7a3b1ac2ac0.jpg'], #'car wheel'
# 402: ['d37fc868c3d574c8c13bb2af71af8b6b.jpg'], #'acoustic guitar'
# 769: ['d13332cf3535b9a55684dadf152a873b.jpg'], #'rule'
# 650: ['24138a8d4e616785c758c55ee4939e7c.jpg','360124ba1848d6bbb16015f3f8f4a433.jpg', 'a541038bf5f90194a281e74ee092ddb3.jpg','b2a9e39106be979e5637270986dcb790.jpg'], #'microphone'
# 890: ['77d1ab677cf95ab0d9775e8937ee39e8.jpg'], #'volleyball'
# 897: ['8ea1a9a37d2552844a30a028342a744c.jpg'], #'washer'
# 530: ['0b95f80b1234b542739c2c6c71a8e0cd.jpg', '6706d99a599829b82fcce3e782cadcaf.jpg'], #'digital clock'
# 599: ['2af5d6459144866693da90e59b3c8c73.jpg', '32da3320229e60728af62940fa56e3b8.jpg', '583b7a272a83d34b261d8ee9a5b8621a.jpg', 'af461e48047e3689841d01713077b39f.jpg'], #'honeycomb'
# 951: ['2f83c20e5d57a920db8c44576dec5c63.jpg','93d7e2a96a5dfcb42c2b2c858c911155.jpg', '30c30e9335411466d3c6491cb3b0e6dc.jpg'], #'lemon'
# 470: ['05a15cd486668115b3b940d60715c791.jpg','a9a3b8277c0acdb84e37073fa706b928.jpg'], #'candle'
# 462: ['ee2093aeaedb9f86ef44d1be73dedc6a.jpg', '17293ce2d956811f5b7b2d3c8bd3bf37.jpg', '1b08936e2b7582117b6bb13ee8382a9f.jpg', '71a5f567b2d2ed3b444bffa513f10d36.jpg'], #'broom'
# 881: ['ef8c58452bfade87c545fcb5733c4867.jpg'], #'upright'
# 417: ['fa0b1aac96ce169b338a1e3f76330381.jpg', 'e4d1fef156a72256fedb57c1484710db.jpg', '282480dba1296d7b3f3d9dabe59921be.jpg'], #'balloon',
# 778: ['b878bdeb4d8ca7b487fe08494795fcfe.jpg'], #'scale',
# 794: ['0436f37e97eb578293cb6682cad1821e.jpg'], #'shower curtain'
# 412: ['717eaef845e3b1c86ccb0b39da53bfb6.jpg', '693c020bf556bf928d64d0e6e3bb8180.jpg', 'bfa87c1c790dc2b94cc47a091c842c15.jpg'],  # ashcan
# 806: ['773e5a0b9645355e37e208f735f327e1.jpg', '40a9736b651208ae73c6aeeea743d6b6.jpg', '0a0027b9155adcdd886761dcbf6a6ece.jpg', '199c216a06fcc4809bbda68226f67343.jpg', '249ed756c7098cc85a67628ddaae9ca5.jpg', '254f6de4acc5a418f4a864f365ad9552.jpg', '2bb38a302e2f550943632b90a33252c4.jpg', '326ee28d1634ae499a976560ec2d5678.jpg', '5c95de4bd9f3305e77315a585530a855.jpg', '73da5ce96158dbd768b6840add4162e8.jpg', '76f7bc909752de61431e9db54b55a3da.jpg', '7d1a3473be44e8a3cadc19bdb438cb46.jpg', '7daa4c7810befdfdfa49d95e0148ba25.jpg', '94416fc9fdc43af1a35fdae411c62491.jpg', '9bcdf2efcddb5159753f054f994eb64a.jpg', '9d2af1789179e3bc125e715dadd4db63.jpg', 'a12b3edfa7603269aa6e6726997a243f.jpg', 'b4bce891965fbad1165a2ada57d3325e.jpg', 'bf57badb33ec4dd212a0f0a129f8f8b7.jpg', 'bf8b7bcb8278faa9ad147443e5509299.jpg', 'c62729d940436b7824151c31ad78b310.jpg', 'd9a479d53496c784b8c0df33d8c0ee30.jpg'], # sock
# 742: ['18fa3fd22e20a4bad2495691cf03819c.jpg','1223de6713144c18226d11d82f1caddd.jpg','a0e14bf5474296f06ef0b46171527e85.jpg','bdfc9ee9926a78ef38c953ad68abc767.jpg'], #'printer'
# 630: ['e01e3755d5ff0cfe28c84545bd6bf65c.jpg', '4752341e2bed74a0e1b5d8e9ce068208.jpg'] #loafer
# }

In [ ]:
# def expand(url):
#     return df1[df1['label_group'] == df1[df1['image'] == url]['label_group'].iloc[0]]['image'].tolist()

# haha = {}
# for x in mycatalog:
#     if x not in haha:
#         haha[x] = []
#     for y in mycatalog[x]:
#         print(y, expand(y))
#         a = expand(y)
#         haha[x] += a
# for x in haha:
#     haha[x] = set(haha[x])

In [ ]:
# for q in haha[471]:
#     plt.imshow(Image.open(f'/kaggle/input/shopee-product-matching/train_images/{q}'))

In [ ]:
# plt.imshow(Image.open(f'/kaggle/input/shopee-product-matching/train_images/92f5aa2dbea16c926c5fb1735d62d08f.jpg'))

In [ ]:
# #VGG16 (imagenet) + VGG16 (coco)

# # model_resnet50_imagenet = ResNet50(weights='/kaggle/input/shopee-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224,224,3))
# # tt = model_resnet50_imagenet.get_layer('conv5_block3_out').output
# # avg_pool = GlobalAveragePooling2D(name='avg_pool')(tt)
# # model_resnet50_conv = Model(inputs=model_resnet50_conv.input, outputs=avg_pool)

# # for layer in model_resnet50_conv.layers:
# #     layer._name += '_'
# #     layer.trainable = False

# # for layer in model_resnet50_conv.layers: 
# #     layer.trainable = False


# last_layer_b4_imagenet = model_b4_imagenet.get_layer('avg_pool').output
# last_layer_resnet50_conv = model_resnet50_conv.get_layer('avg_pool_').output

# concatenate = Concatenate()([last_layer_b4_imagenet, last_layer_resnet50_conv])

# model = Model(inputs=[model_b4_imagenet.input, model_resnet50_conv.input],
#           outputs=concatenate)


# #print(model.summary())

# ins = np.zeros((0,1536+2048))
# batch_vanilla = []
# batch = []
# #import time

# images_vanilla = np.zeros(shape=(1, 300, 300, 3))
# images = np.zeros(shape=(1, 224, 224, 3))

# img = preprocess(f'/kaggle/input/shopee-product-matching/test_images/{testpaths[0]}') 

# images[0, :, :, :] = np.expand_dims(cv2.resize(img, (224, 224)), axis=0)
# images_vanilla[0, :, :, :] = np.expand_dims(img, axis=0)

# batch_vanilla.append(images_vanilla)
# images = ppi_resnet50(images)
# batch.append(images)
# for path in range(1,len(testpaths)):
#     #start = time.time()
#     images_vanilla = np.zeros(shape=(1, 300, 300, 3))
#     images = np.zeros(shape=(1, 224, 224, 3))

#     img = preprocess(f'/kaggle/input/shopee-product-matching/test_images/{testpaths[path]}') 
    
#     images[0, :, :, :] = np.expand_dims(cv2.resize(img, (224, 224)), axis=0)
#     images_vanilla[0, :, :, :] = np.expand_dims(img, axis=0)
    
#     batch_vanilla.append(images_vanilla)
#     images = ppi_resnet50(images)
#     batch.append(images)
#     if path % 32 == 0:
#         batch_vanilla = np.vstack(batch_vanilla)
#         batch = np.vstack(batch)
#         images_features = model.predict([batch_vanilla, batch], batch_size=32)
#         batch_vanilla = []
#         batch = []
# #         print(time.time()-start)
#         ins = np.vstack([ins,images_features])
# if path % 32 != 0:
#     batch_vanilla = np.vstack(batch_vanilla)
#     batch = np.vstack(batch)
#     images_features = model.predict([batch_vanilla, batch], batch_size=path % 32)
# #     print(images_features.shape)
#     batch_vanilla = []
#     batch = []
#     # print(time.time()-start)
#     ins = np.vstack([ins,images_features])

# #     print(len(ins))

In [ ]:
# del images
# del img
# del batch_vanilla
# del batch
# del model_b4_imagenet
# del model_resnet50_imagenet
# del model
# del concatenate
# del EfficientNetB3
# collect()

In [ ]:
# # !wget https://dord.mynetgear.com/modelSelection13.joblib
# skb = load('/kaggle/input/zzxjoanw/goodman_800.joblib')

In [ ]:
# def mergeClusters(cluster_img, cluster_words):
#     cluster_img2 = np.copy(cluster_img)
#     for x in range(np.max(cluster_words)+1):
# #         print(x)
#         tryFind = np.where(cluster_words==x)
#         if len(tryFind[0]) <= 1:
#             continue
# #         print(tryFind[0], cluster_img[tryFind[0]])
#         found = np.copy(cluster_img2[tryFind[0]])
#         valueUsedForMerging = found[0]
#         for y in found[1:]:
#             cluster_img2[cluster_img2==y] = valueUsedForMerging
        
#     return cluster_img2

In [ ]:
# ins = np.array(ins)
# ins.shape
# ss = preprocessing.StandardScaler()
# ins = ss.fit_transform(ins)

In [ ]:
# ins = skb.transform(ins)

# del skb
# collect()
#pca = load('/kaggle/input/shopee-weights/eff_pca1000.joblib')
#pca = decomposition.PCA(n_components=800)
#ins = pca.transform(ins)

In [ ]:
# ss = preprocessing.StandardScaler()
# ins = ss.fit_transform(ins)

#ins = np.hstack([ins,X_trainx])

In [ ]:
pca = load('../input/zzxjoanw/pca_goodman_800.joblib')
ins = pca.transform(ins)

ins = np.hstack([ins,X_trainx])
mms = preprocessing.MinMaxScaler()
ins = mms.fit_transform(ins)

ins = preprocessing.normalize(ins)

In [ ]:
# ins = preprocessing.normalize(ins)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# batch_size = 4096
# cluster_index = 1

# modelAclabels_ = np.zeros(len(ins))

# for x in range(0, int(len(ins)/batch_size)+1):
#     for y in range(x, int(len(ins)/batch_size)+1):
#         cs_matrix = cosine_similarity(ins[x*batch_size:(x+1)*batch_size], ins[y*batch_size:(y+1)*batch_size])

#         for i in range(0, np.shape(cs_matrix)[0]):
#             for j in range(0, np.shape(cs_matrix)[1]):
#                 if cs_matrix[i,j] > 0.49:
#                     if modelAclabels_[y*batch_size+j] == 0:
#                         modelAclabels_[y*batch_size+j] = cluster_index
#             cluster_index += 1

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# cs_matrix = cosine_similarity(ins, ins)

# cluster_index = 1
# modelAclabels_ = np.zeros(len(ins))
# for i in range(0, np.shape(cs_matrix)[0]):
#     for j in range(0, np.shape(cs_matrix)[1]):
#         if cs_matrix[i,j] >0.49:
#             if(modelAclabels_[j]) == 0:
#                 modelAclabels_[j] = cluster_index
#     cluster_index += 1
# #print(myPred)modelAclabels_

In [ ]:
# tt = TweetTokenizer()
# wnl = WordNetLemmatizer()
# ignored = stopwords.words('english')
# ignored += stopwords.words('indonesian')

# def tokenize(desc):
#     desc = unescape(desc)
# #     ret = [x for x in [stemmer.stem(wnl.lemmatize(wnl.lemmatize(tok,'v'),'n')) for tok in tt.tokenize(desc) if tok not in ignored and tok.isascii() and tok.isalpha()] if len(x) > 2]
#     ret = [x for x in [stemmer.stem(wnl.lemmatize(wnl.lemmatize(tok,'v'),'n')) for tok in tt.tokenize(desc) if tok.isascii() and tok.isalpha()] if len(x) > 2]
#     ret = [x for x in ret if ret not in ignored]
    
# #     print(ret)
#     return ret

# tfidf = feature_extraction.text.TfidfVectorizer(tokenizer=tokenize, use_idf=True, max_features=8000)
# X_trainx = tfidf.fit_transform(titles).toarray()

# len(tfidf.vocabulary_)

In [ ]:
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='single', distance_threshold=0.105, compute_distances=False)
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='complete', distance_threshold=0.26, compute_distances=False)
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='complete', distance_threshold=0.09, compute_distances=False)
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='single', distance_threshold=0.02, compute_distances=False)
# modelAc = cluster.Birch(n_clusters=None, threshold=3.5, branching_factor=50)
# modelAc = cluster.Birch(n_clusters=None, threshold=7, branching_factor=50)
# modelAc = cluster.Birch(n_clusters=None, threshold=11.8, branching_factor=50)
# ins = preprocessing.normalize(ins)

modelAc = hdbscan.HDBSCAN(min_cluster_size=2, cluster_selection_epsilon=0.11, core_dist_n_jobs=-1)#,approx_min_span_tree=True, cluster_selection_method='leaf')
modelAclabels_ = modelAc.fit_predict(ins)

# modelAc = cluster.MiniBatchKMeans(n_clusters=int(len(df0)/3))
# modelAc.fit(ins)

# modelTfidf = cluster.AgglomerativeClustering(n_clusters=None, affinity='euclidean', linkage='complete', distance_threshold=0.4, compute_distances=False)
# modelTfidf.fit(X_trainx)

# predictions = mergeClusters(modelAc.labels_, modelTfidf.labels_)
# print(predictions)

In [ ]:
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='single', distance_threshold=0.105, compute_distances=False)
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='complete', distance_threshold=0.26, compute_distances=False)
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='complete', distance_threshold=0.09, compute_distances=False)
# modelAc = cluster.AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='single', distance_threshold=0.02, compute_distances=False)
# modelAc = cluster.Birch(n_clusters=None, threshold=3.5, branching_factor=50)
# modelAc = cluster.Birch(n_clusters=None, threshold=7, branching_factor=50)
# modelAc = cluster.Birch(n_clusters=None, threshold=11.8, branching_factor=50)

# modelAc = hdbscan.HDBSCAN(min_cluster_size=2, cluster_selection_epsilon=11,core_dist_n_jobs=-1)#,approx_min_span_tree=True, cluster_selection_method='leaf')
# modelAclabels_ = modelAc.fit_predict(ins)

# modelAc = cluster.MiniBatchKMeans(n_clusters=int(len(df0)/3))
# modelAc.fit(ins)

# modelTfidf = cluster.AgglomerativeClustering(n_clusters=None, affinity='euclidean', linkage='complete', distance_threshold=0.4, compute_distances=False)
# modelTfidf.fit(X_trainx)

# predictions = mergeClusters(modelAc.labels_, modelTfidf.labels_)
# print(predictions)

In [ ]:
out = []
postingids = np.array(df0['posting_id'])

for i in range(len(modelAclabels_)):
    if modelAclabels_[i] == -1:
        outstring = postingids[i]
    else:
        outstring = ' '.join(postingids[modelAclabels_ == modelAclabels_[i]]) #predictions == predictions[i]])
#     print(outstring)
    out.append(outstring)

In [ ]:
# df0

In [ ]:
df0.insert(1,'matches',out)

In [ ]:
df1 = df0.drop(labels=['image','image_phash','title'], axis=1)

In [ ]:
# df1

In [ ]:
df1.to_csv('submission.csv', index=False)

In [ ]:
# !ls

In [ ]:
# !cat submission.csv